# Musketeer platform workflow - Task creator

In this notebook we will demonstrate the typical workflow in the Musketeer platform for task creators. It involves:
- Creating a username and password (only required initially)
- Listing existing Federated ML tasks
- Creating a new Federated ML task
- Running that task as aggregator

## Load prerequisites

In [ ]:
import sys
sys.path.append("..")
sys.path.append("../demo")
from demo import register, listing, creator, aggregator
from demo import platform_utils as utils

from time import time, gmtime, strftime

import logging
logger = logging.getLogger()
handler = logger.handlers[0]
handler.setLevel(logging.INFO)

## Create a user name and password

In [ ]:
credentials = 'json credentials file>' # e.g. '../local_credential_sample.json'
user = '<user name>' # note local mode requires that user names must be different for aggregator and participants
password = '<password>'
org = '<organization>'

In [ ]:
platform = 'cloud'
context = utils.platform(platform, credentials)

# Only run this if you had not created this user before:
try:        
    register.create_user(context, user, password, org)
except Exception as err:
    print("Error:", err)

## Listing existing Federated ML tasks

In [ ]:
#Now create the platform context for the new user
context = utils.platform(platform, credentials, user, password)

tasks = listing.get_tasks(context)

In [ ]:
print("Number of tasks:", len(tasks))

In [ ]:
# List all tasks that have been added in the last 24 hours:
for task in tasks:
    if task['added'] >= strftime('%Y-%m-%dT%H:%M:%S', gmtime(time() - 3600*24)):
        print(task['task_name'], '\t', task['added'], '\t', task['status'])

## Creating a new Federated ML task

In [ ]:
# Note: task names need to be unique, so you need to use a new name for each task that you create
task_name = '<task name>'

task_definition = {"aggregator": "neural_network.Aggregator",
                   "participant": "neural_network.Participant",
                   "quorum": 1,
                   "round": 2,
                   "epoch": 1,
                   "batch_size": 256,
                   "learning_rate": 0.001,
                   "training_size": 3000,
                   "test_size": 1000,
                  }

In [ ]:
try:
    result = creator.create_task(context, task_name, task_definition)
except Exception as err:
    print("Error:", err)

In [ ]:
tasks = listing.get_tasks(context)

In [ ]:
print('New task available in Musketeer: ', task_name in [t["task_name"] for t in tasks])

In [ ]:
print([t for t in tasks if t["task_name"] == task_name][0])

## Running that task as aggregator

In [ ]:
try:
    aggregator.run(context, task_name)
except Exception as err:
    print("Error:", err)